# 02. Data Merging & Consolidation (Raw Data)
This notebook merges all datasets (Enrolment, Biometric, Demographic) into a single master view.
**Note**: No data cleaning or filtering is performed. All anomalies, duplicates, and missing values are preserved for downstream fraud detection.

In [10]:
import sys
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.preprocessing.data_loader import load_all_datasets
from src.preprocessing.feature_engineering import create_master_table

datasets = load_all_datasets(os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw')))
print("Datasets loaded.")

Loading enrolment from split folder: api_data_aadhar_enrolment
Found 3 files in c:\Users\Kaustab das\Desktop\Aadhaar Pulse AI\api_data_aadhar_enrolment. Merging...
Loading biometric from split folder: api_data_aadhar_biometric
Found 4 files in c:\Users\Kaustab das\Desktop\Aadhaar Pulse AI\api_data_aadhar_biometric. Merging...
Loading demographic from split folder: api_data_aadhar_demographic
Found 5 files in c:\Users\Kaustab das\Desktop\Aadhaar Pulse AI\api_data_aadhar_demographic. Merging...
Datasets loaded.


## 1. Merge Datasets
We will merge Enrolment, Biometric, and Demographic data into a single master table keyed by `Date`, `State`, `District`, and `Pincode`.

In [11]:
master_df = create_master_table(datasets)
print("Master Table Shape:", master_df.shape)
display(master_df.head())

Master Table Shape: (2947681, 15)


,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,bio_age_5_17,bio_age_17_,demo_age_5_17,demo_age_17_,total_enrolment,total_bio_updates,total_demo_updates,total_updates
0,2025-03-02,Meghalaya,East Khasi Hills,793121,11.0,61.0,37.0,NaN,NaN,NaN,NaN,109.0,0.0,0.0,0.0
1,2025-03-09,Karnataka,Bengaluru Urban,560043,14.0,33.0,39.0,NaN,NaN,NaN,NaN,86.0,0.0,0.0,0.0
2,2025-03-09,Uttar Pradesh,Kanpur Nagar,208001,29.0,82.0,12.0,NaN,NaN,NaN,NaN,123.0,0.0,0.0,0.0
3,2025-03-09,Uttar Pradesh,Aligarh,202133,62.0,29.0,15.0,NaN,NaN,NaN,NaN,106.0,0.0,0.0,0.0
4,2025-03-09,Karnataka,Bengaluru Urban,560016,14.0,16.0,21.0,NaN,NaN,NaN,NaN,51.0,0.0,0.0,0.0


## 2. Integrity Check (Duplicates & Anomalies)
We identity duplicate records (same district/date) and missing values. 
**Action**: These are **NOT** removed. They are flagged as potential fraud signals.

In [12]:
# Check duplicates (Potential Double-Counting Fraud)
duplicates = master_df.duplicated(subset=['date', 'state', 'district', 'pincode'])
print(f"Duplicate records found (Potential Anomalies): {duplicates.sum()}")

# NOTE: We strictly DO NOT drop duplicates. 
# They are preserved to analyze reporting inconsistencies.

# Check for missing values (preserved for fraud/anomaly detection)
print("Missing values (preserved - 0 means full data, numbers indicate missing source reports):")
print(master_df.isnull().sum())

Duplicate records found (Potential Anomalies): 617213
Missing values (preserved - 0 means full data, numbers indicate missing source reports):
date                        0
state                       0
district                    0
pincode                     0
age_0_5               1697241
age_5_17              1697241
age_18_greater        1697241
bio_age_5_17           694666
bio_age_17_            694666
demo_age_5_17          770073
demo_age_17_           770073
total_enrolment             0
total_bio_updates           0
total_demo_updates          0
total_updates               0
dtype: int64


## 3. Save Processed Data
Saving the master table for further analysis.

In [13]:
processed_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'processed'))
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
output_path = os.path.join(processed_dir, 'merged_master_table.csv')

master_df.to_csv(output_path, index=False)
print(f"Saved RAW merged master table to: {output_path}")

Saved RAW merged master table to: c:\Users\Kaustab das\Desktop\Aadhaar Pulse AI\data\processed\merged_master_table.csv
